# Conexión de capas de atención y capas lineales en un bloque transformador

En  esta  sección,  se implementará  el  bloque  transformador,  un  componente  fundamental  de  GPT  y  otras  arquitecturas  LLM.  Este  bloque,  que  se  repite  una  docena  de  veces  en  la  arquitectura  GPT2  de  124  millones  de  parámetros,  combina  varios  conceptos  que  ya  hemos  abordado:  atención  multicabezal,  normalización  de  capas,  abandono,  capas  de  avance  y  activaciones  GELU.

![Texto alternativo](./imgs/4.13.png)

El  bloque  transformador  combina  varios  componentes,  incluido  el  módulo  de  atención  de  múltiples  cabezas  enmascaradas  y  el  módulo  FeedForward.

Cuando  un  bloque  transformador  procesa  una  secuencia  de  entrada,  cada  elemento  de  la  secuencia  (por  ejemplo,  un  token  de  palabra  o  subpalabra)  se  representa  mediante  un  vector  de  tamaño  fijo  (768  dimensiones).  Las  operaciones  dentro  del  bloque  transformador,  incluyendo  las  capas  de  atención  multicabezal  y  de  avance,  están  diseñadas  para  transformar  estos  vectores  de  forma  que  se  preserve  su  dimensionalidad.

La  idea  es  que  el  mecanismo  de  autoatención  en  el  bloque  de  atención  de  múltiples  cabezas  identifique y  analiza  las  relaciones  entre  los  elementos  de  la  secuencia  de  entrada.  En  cambio,  la  red  de  propagación  hacia  adelante  modifica  los  datos  individualmente  en  cada  posición.  Esta  combinación  no  solo  permite  una  comprensión  y  un  procesamiento  más  precisos  de  la  entrada,  sino  que  también  mejora  la  capacidad  general  del  modelo  para  gestionar  patrones  de  datos  complejos.

#### Paso 1: Convertir las palabras en perfiles numéricos (Vectores)

Primero, la máquina no entiende de palabras como "gato" o "casa". Necesita convertirlas en algo que pueda procesar: números.

* Cada palabra (o trozo de palabra, llamado **token**) se convierte en un **vector**: una larga lista de números (en tu ejemplo, 768 números).
* Piensa en este vector como un **perfil o ficha de identidad súper detallado** de la palabra. Esta ficha no solo dice qué palabra es, sino que también captura su significado, su función gramatical y sus posibles contextos. Por ejemplo, los vectores de "rey" y "reina" serían muy parecidos entre sí.

> **Clave:** Todas las "fichas" (vectores) tienen el mismo tamaño (768 dimensiones), lo que permite que la máquina las procese de manera uniforme.

---

#### Paso 2: El trabajo dentro del bloque (Los dos especialistas)

Una vez que todas las palabras de la frase son "fichas numéricas", entran en el bloque transformador. Dentro, hay dos especialistas que trabajan en equipo:

##### 1. La Capa de Atención Multicabezal: El Analista de Contexto 

Este es el especialista en **relaciones**. Su trabajo es mirar todas las palabras de la frase a la vez y averiguar cómo se conectan entre sí.

* **Ejemplo:** En la frase "El robot cogió la manzana porque estaba madura", la palabra "madura" se refiere a la "manzana", no al "robot".
* El mecanismo de **atención** es como un detective que traza líneas de conexión, dándole más importancia a la relación *"manzana"* <-> *"madura"* y muy poca a *"robot"* <-> *"madura"*.
* Lo de **"multicabezal"** (*multi-head*) significa que no hay un solo detective, sino varios. Cada uno se especializa en buscar un tipo de relación diferente (uno busca sujeto-verbo, otro adjetivo-sustantivo, etc.).

Al final de su trabajo, este especialista actualiza la "ficha" (el vector) de cada palabra, añadiéndole toda esta nueva información sobre su contexto y sus relaciones importantes en la frase.

##### 2. La Red de Propagación Hacia Adelante: El Procesador Individual 

Después de que el primer especialista ha añadido el contexto, entra en juego el segundo. Este es un especialista que trabaja **individualmente** con cada palabra.

* Toma la "ficha" ya enriquecida con el contexto y la procesa por su cuenta.
* Su función es "digerir" esa nueva información y refinar el significado de esa palabra específica. Es como si, después de una reunión de equipo (la atención), cada miembro fuera a su escritorio a pensar profundamente sobre la información recibida y a hacer su propio trabajo.

Este paso añade más capacidad de aprendizaje y permite al modelo captar conceptos más complejos y abstractos sobre cada elemento.

---

### ¿Por qué esta combinación es tan potente?

La magia está en combinar estos dos pasos:

1.  Primero, la **atención** mira el cuadro completo y entiende las **relaciones entre las piezas** (el contexto).
2.  Luego, la **propagación hacia adelante** se enfoca en **cada pieza individualmente** para procesar y refinar su significado con ese nuevo contexto.

Esta combinación permite que el modelo entienda no solo lo que significa cada palabra por sí sola, sino, y más importante, lo que significa dentro del tejido complejo de una frase. Al repetir este proceso en muchos bloques apilados, el modelo consigue una comprensión del lenguaje increíblemente profunda y matizada.

In [16]:
from multiHeadAttention import MultiHeadAttention
from feedForward import FeedForward
from layerNorm import LayerNorm
import torch
import torch.nn as nn

GPT_CONFIG_124M = {
    "vocab_size": 50257,  
    "context_length": 1024,     
    "emb_dim": 768,     
    "n_heads": 12,        
    "n_layers": 12,   
    "drop_rate": 0.1,    
    "qkv_bias": False   
}

class TransformerBlock(nn.Module):
    def __init__(self, cfg=GPT_CONFIG_124M):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            dropout=cfg["drop_rate"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            qkv_bias=cfg["qkv_bias"]
        )
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):   
        ## Bloque 1 atencion
        shortcut = x    #conexion de acceso directo para el bloque de atencion
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut 

        #Bloque 2: Red neuronal feed_forward
        shortcut = x    #conexion de acceso directo para el bloque de avence
        x = self.norm2(x)   
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut    #agregaci´on de entrada principal
        return x





### Diagrama del Flujo de Datos

```text
Entrada (x)
   ↓
┌──────────────────────────────┐
│   Sub-bloque 1: Atención     │
│------------------------------│
│ 1. shortcut = x              │
│ 2. norm1(x)                  │
│ 3. att(x)                    │
│ 4. drop_shortcut(x)          │
│ 5. x = x + shortcut          │
└──────────────────────────────┘
   ↓
┌──────────────────────────────┐
│ Sub-bloque 2: Feed-Forward   │
│------------------------------│
│ 1. shortcut = x              │
│ 2. norm2(x)                  │
│ 3. ff(x)                     │
│ 4. drop_shortcut(x)          │
│ 5. x = x + shortcut          │
└──────────────────────────────┘
   ↓
Salida
```

### Explicación de las Partes Clave
1. Los Dos "Especialistas" Principales
self.att = MultiHeadAttention(...): Este es el analista de contexto. Su única misión es mirar todos los vectores de la secuencia y recalcular cada uno de ellos para que contenga información sobre las palabras relevantes a su alrededor. Es el que decide que "la" en "la gata" se refiere a "gata".

   self.ff = FeedForward(...): Este es el procesador individual. Una vez que la capa de atención ha enriquecido cada vector con su contexto, esta red neuronal simple procesa cada vector de forma aislada para extraer patrones más complejos y refinar su significado.

2. Los Componentes de Soporte (¡Igual de importantes!)
Normalización de Capa (self.norm1 y self.norm2): Piensa en esto como un "control de volumen". A medida que los datos pasan por muchas capas, los números de los vectores pueden volverse muy grandes o muy pequeños, lo que desestabiliza el aprendizaje. La normalización los mantiene en un rango "saludable", haciendo que el entrenamiento sea mucho más estable y rápido. Tu código usa una arquitectura Pre-LN (Pre-LayerNorm), donde la normalización se aplica antes de la atención y la red neuronal, lo cual es muy común y efectivo.

   Conexiones Residuales (x = x + shortcut): Esta es una de las ideas más importantes. Al sumar la entrada original (shortcut) a la salida procesada, te aseguras de que el bloque no "olvide" la información original. Permite que la información fluya directamente a través del bloque sin ser modificada. Esto es crucial para poder entrenar modelos con muchísimas capas (cientos de bloques), ya que combate el problema de la "desaparición del gradiente". Es como una autopista que permite a la información original saltarse el procesamiento si es necesario.

   Dropout (self.drop_shortcut): Esta es una técnica para evitar que el modelo "memorice" los datos de entrenamiento en lugar de aprender patrones generales. Durante el entrenamiento, apaga aleatoriamente algunas neuronas. Esto obliga al resto de la red a aprender de una forma más robusta y a no depender de unas pocas neuronas específicas.

In [17]:
torch.manual_seed(123)
x = torch.rand(2, 4, 768) #[batch_size, num_tokens, emb_dim]
block = TransformerBlock(GPT_CONFIG_124M)
output = block(x)

print("Input shape:", x.shape)
print("Output shape:", output.shape)

Input shape: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])


![Texto alternativo](./imgs/4.14.png)

El  bloque  transformador  combina  la  normalización  de  capas,  la  red  de  alimentación  hacia  adelante  (incluidas  las  activaciones  GELU)  y  las  conexiones  de  acceso  directo. Este  bloque  transformador  constituirá  el  componente  principal  de  la  arquitectura  GPT  que se implementará.

[Codificación del modelo GPT](./6_codificacion_modelo_gpt.ipynb)